# Protocol Buffer Basics: Python
This tutorial provides a basic Python programmer's introduction to working with protocol buffers. By walking through creating a simple example application, it shows you how to:
* Define message formats in a `.proto` file.
* Use the protocol buffer compiler.
* Use the Python protocol buffer API to write and read messages.

## Defining Your Protocol Format
First create a `.proto` file name it : `addressbook.proto`

```sh
nano addressbook.proto
```
`
write the bellow message structure into the file:

syntax = "proto2";

package tutorial;

message Person {
  required string name = 1;
  required int32 id = 2;
  optional string email = 3;

  enum PhoneType {
    MOBILE = 0;
    HOME = 1;
    WORK = 2;
  }

  message PhoneNumber {
    required string number = 1;
    optional PhoneType type = 2 [default = HOME];
  }

  repeated PhoneNumber phones = 4;
}

message AddressBook {
  repeated Person people = 1;
}
`

Save the file, the `.proto` file starts with a package declaration, which helps to prevent naming conflicts between different projects. In Python, packages are normally determined by directory structure, so the package you define in your `.proto` file will have no effect on the generated code. However, you should still declare one to avoid name collisions in the Protocol Buffers name space as well as in non-Python languages.

Next, you have your message definitions. A message is just an aggregate containing a set of typed fields. Many standard simple data types are available as field types, including `bool`, `int32`, `float`, `double`, and `string`. You can also add further structure to your messages by using other message types as field types – in the above example the Person message contains PhoneNumber messages, while the AddressBook message contains Person messages. You can even define message types nested inside other messages – as you can see, the PhoneNumber type is defined inside Person. You can also define enum types if you want one of your fields to have one of a predefined list of values – here you want to specify that a phone number can be one of MOBILE, HOME, or WORK.

The " = 1", " = 2" markers on each element identify the unique "tag" that field uses in the binary encoding. Each field must be annotated with one of the following modifiers:
* `required`: a value for the field must be provided, otherwise the message will be considered "uninitialized". Serializing an uninitialized message will raise an exception. Parsing an uninitialized message will fail. Other than this, a required field behaves exactly like an optional field.
* `optional`: the field may or may not be set. If an optional field value isn't set, a default value is used. For simple types, you can specify your own default value, as we've done for the phone number type in the example. Otherwise, a system default is used: zero for numeric types, the empty string for strings, false for bools. For embedded messages, the default value is always the "default instance" or "prototype" of the message, which has none of its fields set. Calling the accessor to get the value of an optional (or required) field which has not been explicitly set always returns that field's default value.
* `repeated`: the field may be repeated any number of times (including zero). The order of the repeated values will be preserved in the protocol buffer. Think of repeated fields as dynamically sized arrays.

## Compiling Your Protocol Buffers

First of all you must have Protobuf installed in your computer if not run :
```sh
sudo apt-get install protobuf-compiler
sudo pip3 install --upgrade protobuf
```
after that you can compile your file:
```sh
SRC_DIR=/home/sanji/projets/protobuf_training/protobufdemo
DST_DIR=/home/sanji/projets/protobuf_training/protobufdemo
protoc -I=$SRC_DIR --python_out=$DST_DIR $SRC_DIR/addressbook.proto
```
A file named **addressbook_pb2.py** will be created, in my case the `addressbook.proto` file and `addressbook_pb2.py` are in a subdirectory named `protobufdemo`, we put also in this subdirecotry a file named `__init__.py` so lets use the generated `Person` class

## Use the Python protocol buffer API to write and read messages

In [1]:
from protobufdemo import addressbook_pb2
person = addressbook_pb2.Person()
person.id = 1234
person.name = "RABBAH Mahmoud"
person.email = "mrabbah@gmail.com"
phone = person.phones.add()
phone.number = "555-4321"
phone.type = addressbook_pb2.Person.HOME

In [3]:
print(person)

name: "RABBAH Mahmoud"
id: 1234
email: "mrabbah@gmail.com"
phones {
  number: "555-4321"
  type: HOME
}



In [5]:
# Parsing and Serialization
s_serialized_p = person.SerializeToString()
print(s_serialized_p)

b'\n\x0eRABBAH Mahmoud\x10\xd2\t\x1a\x11mrabbah@gmail.com"\x0c\n\x08555-4321\x10\x01'


In [6]:
person_2 = addressbook_pb2.Person()
person_2.ParseFromString(s_serialized_p)
print(person_2)

name: "RABBAH Mahmoud"
id: 1234
email: "mrabbah@gmail.com"
phones {
  number: "555-4321"
  type: HOME
}



In [7]:
# Write to a file serialized message
f = open('protobufdemo/data.raw', "wb")
f.write(person_2.SerializeToString())
f.close()

In [8]:
# Read from a file serialized message
f2 = open('protobufdemo/data.raw', "rb")
person_3 = addressbook_pb2.Person()
person_3.ParseFromString(f2.read())
f2.close()
print(person_3)

name: "RABBAH Mahmoud"
id: 1234
email: "mrabbah@gmail.com"
phones {
  number: "555-4321"
  type: HOME
}

